In [ ]:
# !wget https://raw.githubusercontent.com/allenai/bi-att-flow/master/squad/evaluate-v1.1.py
# !pip install sentencepiece
# !pip install peft
# !pip install sacrebleu
# !git clone https://github.com/google-deepmind/xquad

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline, AutoModelWithLMHead
import torch
from  transformers  import  AutoTokenizer, AutoModelWithLMHead, pipeline
from google.colab import drive
import glob
from transformers.pipelines import pipeline
from transformers import AutoModelForQuestionAnswering
from transformers import AutoTokenizer
import os
import re
import json
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [ ]:
def do_experiment(path_regex,pipe,test_fn, result_path, prefix, batch_size):
    for file_path in glob.glob(path_regex):
        file_name = os.path.split(file_path)[-1]
        lang = file_name.split('.')[1]
        results = []
        with open(file_path, 'r', encoding='utf-8') as f:
            xquad_json = json.load(f)
            for data in xquad_json['data']:
                for paragraph in data['paragraphs']:
                  context = paragraph['context']
                  questions = list(map(lambda x:x['question'], paragraph['qas']))
                  ids = list(map(lambda x:x['id'], paragraph['qas']))
                  for batch_question, batch_ids in zip(batch(questions, batch_size), batch(ids, batch_size)):
                      model_answers = test_fn(context, batch_question, pipe)
                      for model_answer, id in zip(model_answers, batch_ids):
                          results.append({id:model_answer})

        out_path = os.path.join(result_path, prefix + '/' + file_name)
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with open(out_path, "w+", encoding='utf-8') as out_f:
            json.dump(results, out_f, ensure_ascii=False)

In [ ]:
model_name = "alon-albalak/bert-base-multilingual-xquad"

pipe = pipeline('question-answering', model=model_name, tokenizer=model_name, device='cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def test_function(context, questions, pipe):
    input = [{'question': question,'context': context} for question in questions]
    output = pipe(input, batch_size=len(input))
    if isinstance(output, list):
       return [x['answer'] for x in output]
    return [output['answer']]

In [ ]:
path_regex = '/content/drive/MyDrive/xquad.??.json'
do_experiment(path_regex,pipe,test_function, '/content/drive/MyDrive/', 'bert_csw', 32)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
